In [2]:
import pandas as pd
import numpy as np
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import urllib
import requests
from PyPDF2 import PdfMerger
from urllib.parse import urlparse
from io import BytesIO
from enum import Enum
import re
from urllib.error import URLError, HTTPError  # Import these classes from urllib.error
import ssl
import pymupdf
import os

## Extract Link from 150 Good research paper

In [4]:
# Read in the csv file
## This is a csv file that teammates manually inspected as good research pape
good_df = pd.read_csv("../data/150_research_papers.csv")[['Link']]
good_df['label'] = 1

In [5]:
good_df.head()
# good_df.shape

,Link,label
0,https://www.science.org/doi/10.1126/science.ad...,1
1,https://www.nature.com/articles/s41566-019-0398-2,1
2,https://www.nature.com/articles/s41560-020-007...,1
3,https://www.science.org/doi/10.1126/science.ab...,1
4,https://www.nature.com/articles/s41467-021-236...,1


## Import bad paper

In [7]:
bad_df = pd.read_csv("../data/irrelevant_papers.txt")
bad_df.head()

,link
0,https://doi.org/10.1016/0021-9797(77)90150-3
1,https://www.sciencedirect.com/science/article/...
2,https://www.sciencedirect.com/science/article/...
3,https://doi.org/10.1107%2FS0021889813019535
4,https://www.sciencedirect.com/science/article/...


In [8]:
bad_df['label'] = 0
# sampled_bad_df.to_csv('bad_paper_small.csv', index=False)

In [9]:
bad_df

,link,label
0,https://doi.org/10.1016/0021-9797(77)90150-3,0
1,https://www.sciencedirect.com/science/article/...,0
2,https://www.sciencedirect.com/science/article/...,0
3,https://doi.org/10.1107%2FS0021889813019535,0
4,https://www.sciencedirect.com/science/article/...,0
...,...,...
310,https://www.sciencedirect.com/science/article/...,0
311,https://www.sciencedirect.com/science/article/...,0
312,https://www.sciencedirect.com/science/article/...,0
313,https://doi.org/10.1063/1.1404988,0


# Scraping Text

In [10]:
def get_text(soup):
    #Given a beautiful soup object, it will extact the text
    for script in soup(['script', 'style']):
        script.extract()
    text = soup.get_text(separator=' ')
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)
    return text

In [11]:
def create_txt_list(df, col_name):
    #Function that given dataframe and column name that stores the link, 
    #We will extract the text and store them into a list
    #This function returns a modified df where they add a new column "text" associated with an list. 
    text_lst = []
    page_count = 0
    for link in df[col_name]:
        page_count += 1
        print(f"Analyzing {page_count}th link")
        try:
            page = urlopen(link)
            print("Page accessed successfully.")
        except HTTPError as e:
            print(f"HTTP error occurred: {e.code} - {e.reason}")
            if e.code == 403 and e.reason == 'Forbidden':
                print("Found a forbidden access exception ")
                hdr = {'User-Agent': 'Mozilla/5.0'}
                try:
                    # Retrying with modified request headers
                    request = Request(link, headers=hdr)
                    page = urlopen(request)
                    print("Page accessed successfully with headers.")
                except HTTPError as retry_e:
                    print(f"Retry HTTP error occurred: {retry_e.code} - {retry_e.reason}")
                    text_lst.append("HTTP Error, No Access")
                    continue
                except URLError as retry_e:
                    print(f"Retry URL error occurred: {retry_e.reason}")
                    text_lst.append("URL Error, No Access")
                    continue
            else:
                print("NOT a forbidden access exception ")
                text_lst.append("HTTP Error, No Access")
                continue
        except URLError as e:
            print(f"URL error occurred: {e.reason}")
            text_lst.append("URL Error, No Access")
            continue
        except ssl.SSLError as e:
            print(f"SSL error occurred: {e}")
            text_lst.append("SSL Error, No Access")
            continue
        except ValueError as e:
            print(f"Value error (likely an invalid URL): {e}")
            text_lst.append("Invalid URL No Access")
            continue
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            text_lst.append("Unexpected Error No Access")
            continue
        html_content = page.read().decode("utf-8")
        soup = BeautifulSoup(html_content, "html.parser")
        text_lst.append(get_text(soup))
        print()
    df['text'] = text_lst
    return df

In [13]:
#Extract text from good text dataframe
bad_df_text = create_txt_list(bad_df, "link")

Analyzing 1th link
Page accessed successfully.

Analyzing 2th link
HTTP error occurred: 403 - Forbidden
Found a forbidden access exception 
Retry HTTP error occurred: 403 - Forbidden
Analyzing 3th link
HTTP error occurred: 403 - Forbidden
Found a forbidden access exception 
Retry HTTP error occurred: 403 - Forbidden
Analyzing 4th link
Page accessed successfully.

Analyzing 5th link
HTTP error occurred: 403 - Forbidden
Found a forbidden access exception 
Retry HTTP error occurred: 403 - Forbidden
Analyzing 6th link
HTTP error occurred: 403 - Forbidden
Found a forbidden access exception 
Page accessed successfully with headers.

Analyzing 7th link
HTTP error occurred: 403 - Forbidden
Found a forbidden access exception 
Retry HTTP error occurred: 403 - Forbidden
Analyzing 8th link
Page accessed successfully.

Analyzing 9th link
HTTP error occurred: 403 - Forbidden
Found a forbidden access exception 
Retry HTTP error occurred: 403 - Forbidden
Analyzing 10th link
Page accessed successfully.

In [14]:
bad_df_text.to_csv('../data/text_bad_paper.csv', index=False)

In [15]:
#Extract text from good dataframe
good_df_text = create_txt_list(good_df, "Link")

Analyzing 1th link
HTTP error occurred: 403 - Forbidden
Found a forbidden access exception 
Retry HTTP error occurred: 403 - Forbidden
Analyzing 2th link
Page accessed successfully.

Analyzing 3th link
Page accessed successfully.

Analyzing 4th link
HTTP error occurred: 403 - Forbidden
Found a forbidden access exception 
Retry HTTP error occurred: 403 - Forbidden
Analyzing 5th link
Page accessed successfully.

Analyzing 6th link
HTTP error occurred: 403 - Forbidden
Found a forbidden access exception 
Retry HTTP error occurred: 403 - Forbidden
Analyzing 7th link
HTTP error occurred: 403 - Forbidden
Found a forbidden access exception 
Retry HTTP error occurred: 403 - Forbidden
Analyzing 8th link
HTTP error occurred: 403 - Forbidden
Found a forbidden access exception 
Retry HTTP error occurred: 403 - Forbidden
Analyzing 9th link
HTTP error occurred: 403 - Forbidden
Found a forbidden access exception 
Retry HTTP error occurred: 403 - Forbidden
Analyzing 10th link
HTTP error occurred: 403 -

In [16]:
good_df_text.to_csv('../data/text_good_paper.csv', index=False)

## Head of good_df_text and bad_df_text

In [17]:
good_df_text

,Link,label,text
0,https://www.science.org/doi/10.1126/science.ad...,1,"HTTP Error, No Access"
1,https://www.nature.com/articles/s41566-019-0398-2,1,Surface passivation of perovskite film for eff...
2,https://www.nature.com/articles/s41560-020-007...,1,Intact 2D/3D halide junction perovskite solar ...
3,https://www.science.org/doi/10.1126/science.ab...,1,"HTTP Error, No Access"
4,https://www.nature.com/articles/s41467-021-236...,1,Multication perovskite 2D/3D interfaces form v...
...,...,...,...
146,https://doi.org/10.1002%2Fsmll.201803350,1,"HTTP Error, No Access"
147,https://doi.org/10.1038%2Fs41467-022-34203-x,1,Overcoming C60-induced interfacial recombinati...
148,https://doi.org/10.1038%2Fs41560-023-01249-0,1,Bifunctional hole-shuttle molecule for improve...
149,https://doi.org/10.1002%2Fadma.201907757,1,"HTTP Error, No Access"


In [18]:
bad_df_text

,link,label,text
0,https://doi.org/10.1016/0021-9797(77)90150-3,0,Redirecting
1,https://www.sciencedirect.com/science/article/...,0,"HTTP Error, No Access"
2,https://www.sciencedirect.com/science/article/...,0,"HTTP Error, No Access"
3,https://doi.org/10.1107%2FS0021889813019535,0,(IUCr) X-ray analysis of residual stress gradi...
4,https://www.sciencedirect.com/science/article/...,0,"HTTP Error, No Access"
...,...,...,...
310,https://www.sciencedirect.com/science/article/...,0,"HTTP Error, No Access"
311,https://www.sciencedirect.com/science/article/...,0,"HTTP Error, No Access"
312,https://www.sciencedirect.com/science/article/...,0,"HTTP Error, No Access"
313,https://doi.org/10.1063/1.1404988,0,The charge of glass and silica surfaces | The ...


## Filter out the non accessible texts

In [19]:
pass_bad_df = bad_df_text[(bad_df_text['text'] != "HTTP Error, No Access") & (bad_df_text['text'] != "Redirecting")]
pass_bad_df

,link,label,text
3,https://doi.org/10.1107%2FS0021889813019535,0,(IUCr) X-ray analysis of residual stress gradi...
5,https://doi.org/10.1063%2F1.4864778,0,Unusual defect physics in CH3NH3PbI3 perovskit...
9,https://doi.org/10.1039/D3TA00873H,0,Molecular interaction modulating Ruddlesden–Po...
12,https://doi.org/10.1039/C5CE01014D,0,Fundamental growth principles of colloidal met...
14,https://doi.org/10.1038%2Fnatrevmats.2018.17,0,Scalable fabrication of perovskite solar cells...
...,...,...,...
290,https://doi.org/10.1017/9781108394826.004,0,Methods of Colloidal Simulation (Chapter 3) - ...
292,https://doi.org/10.1146/annurev.ms.12.080182.0...,0,Transition Metal Oxide Gels and Colloids | Ann...
300,https://doi.org/10.1039/DC9786500007,0,"The First Rideal Lecture. Microemulsions, a fi..."
302,https://doi.org/10.1038%2Fnphoton.2012.285,0,Strain-engineered artificial atom as a broad-s...


In [20]:
pass_good_df = good_df_text[(good_df_text['text'] != "HTTP Error, No Access") & (good_df_text['text'] != "Redirecting")]
pass_good_df = pass_good_df.rename(columns={'Link': 'link'})
pass_good_df

,link,label,text
1,https://www.nature.com/articles/s41566-019-0398-2,1,Surface passivation of perovskite film for eff...
2,https://www.nature.com/articles/s41560-020-007...,1,Intact 2D/3D halide junction perovskite solar ...
4,https://www.nature.com/articles/s41467-021-236...,1,Multication perovskite 2D/3D interfaces form v...
10,https://doi.org/10.1038%2Fs41586-022-04604-5,1,Stability-limiting heterointerfaces of perovsk...
15,https://doi.org/10.1038%2Fs41467-022-30426-0,1,Imaging and quantifying non-radiative losses a...
...,...,...,...
141,https://doi.org/10.1038%2Fs41586-023-06637-w,1,Anion–π interactions suppress phase impurities...
145,https://doi.org/10.1039%2FD2EE00288D,1,Optimized carrier extraction at interfaces for...
147,https://doi.org/10.1038%2Fs41467-022-34203-x,1,Overcoming C60-induced interfacial recombinati...
148,https://doi.org/10.1038%2Fs41560-023-01249-0,1,Bifunctional hole-shuttle molecule for improve...


## Merge two df together

In [21]:
merged_df = pass_good_df.append(pass_bad_df, ignore_index=True)
merged_df

C:\Users\nicco\AppData\Local\Temp\ipykernel_16452\2237511184.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df = pass_good_df.append(pass_bad_df, ignore_index=True)


,link,label,text
0,https://www.nature.com/articles/s41566-019-0398-2,1,Surface passivation of perovskite film for eff...
1,https://www.nature.com/articles/s41560-020-007...,1,Intact 2D/3D halide junction perovskite solar ...
2,https://www.nature.com/articles/s41467-021-236...,1,Multication perovskite 2D/3D interfaces form v...
3,https://doi.org/10.1038%2Fs41586-022-04604-5,1,Stability-limiting heterointerfaces of perovsk...
4,https://doi.org/10.1038%2Fs41467-022-30426-0,1,Imaging and quantifying non-radiative losses a...
...,...,...,...
127,https://doi.org/10.1017/9781108394826.004,0,Methods of Colloidal Simulation (Chapter 3) - ...
128,https://doi.org/10.1146/annurev.ms.12.080182.0...,0,Transition Metal Oxide Gels and Colloids | Ann...
129,https://doi.org/10.1039/DC9786500007,0,"The First Rideal Lecture. Microemulsions, a fi..."
130,https://doi.org/10.1038%2Fnphoton.2012.285,0,Strain-engineered artificial atom as a broad-s...


In [22]:
merged_df.to_csv('merged_label.csv', index=False)

## Making PDFs of each paper

In [23]:
def get_base_url(url):
    parsed = urlparse(url)
    return f"{parsed.scheme}://{parsed.netloc}"


In [30]:
def get_page(url):
    req = urllib.request.Request(url)
    req.add_header('User-Agent', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:106.0) Gecko/20100101 Firefox/106.0')
    req.add_header('Accept', 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8')
    req.add_header('Accept-Language', 'en-US,en;q=0.5')

    r = urllib.request.urlopen(req)
    return r

In [79]:
def get_actual_url(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}
    r = requests.get(url, headers=headers, allow_redirects=True)
    parsed_uri = urlparse(r.url)
    print(r.url)
    return f'{parsed_uri.scheme}://{parsed_uri.netloc}/'

In [58]:
def reset_eof(pdf_content):
    # find the line position of the EOF
    EOF_MARKER = b'%%EOF'
    if EOF_MARKER in pdf_content:
        # we can remove the early %%EOF and put it at the end of the file
        pdf_content = pdf_content.replace(EOF_MARKER, b'')
        pdf_content = pdf_content + EOF_MARKER
    else:
        # Some files really don't have an EOF marker
        # printed b'\n%%EO%E'
        pdf_content = pdf_content[:-6] + EOF_MARKER
    return pdf_content

In [102]:
def download_all_pdfs(url, paper_index):
    page = get_page(url)
    url = get_actual_url(url)
    print(url)
    soup = BeautifulSoup(page.read().decode("utf-8"), 'html.parser')
    
    # Find all unique PDF links on the page
    pdf_links = [link.get('href') for link in soup.find_all('a', href=True) \
                 if link.get('href') and 'pdf' in urlparse(link.get('href')).path.lower()]
    pdf_links = list(set(pdf_links))
    
    # Initialize a new PDF document to store the merged pages
    merged_pdf = pymupdf.open()
    i = 0
    print(pdf_links)
    for pdf_link in pdf_links:
        # Ensure each link is a full URL
        pdf_url = pdf_link if pdf_link.startswith('http') else get_base_url(url) + pdf_link
        print(pdf_url)
        try:
            res = requests.get(pdf_url, allow_redirects=True)
            print(f"Processing PDF URL: {res.url}")
        except:
            print(f"Skipping invalid PDF at {pdf_url}")
            continue
        
        # Load PDF content into a PyMuPDF document
        try:
            pdf_stream = BytesIO(res.content)
            pdf_document = pymupdf.open(stream=pdf_stream, filetype="pdf")
            
            # Append each page to the merged PDF
            for page_num in range(pdf_document.page_count):
                merged_pdf.insert_pdf(pdf_document, from_page=page_num, to_page=page_num)
                
            pdf_document.close()
            i += 1
        except Exception as e:
            print(f"Skipping invalid PDF at {pdf_url}: {e}")
    
    # Save the merged PDF to disk
    if (merged_pdf.page_count > 0):
        merged_pdf.save(f'../data/pdfs/{paper_index}.pdf')
        merged_pdf.close()
        print(f"Merged PDF saved as '../data/pdfs/{paper_index}.pdf'")
    else:
        print(f"No PDF links found for index {paper_index}.")
        #TODO : use selenium to download the pdf by printing

In [ ]:
for index, row in merged_df.iterrows():
    print(row['link'])
    download_all_pdfs(row['link'], index)

https://doi.org/10.1039/C9CC05490A
https://pubs.rsc.org/en/content/articlelanding/2019/cc/c9cc05490a
https://pubs.rsc.org/
['https://www.rsc.org/suppdata/c9/cc/c9cc05490a/c9cc05490a1.pdf', '/en/content/getauthorversionpdf/C9CC05490A']
https://www.rsc.org/suppdata/c9/cc/c9cc05490a/c9cc05490a1.pdf
Processing PDF URL: https://www.rsc.org/suppdata/c9/cc/c9cc05490a/c9cc05490a1.pdf
https://pubs.rsc.org/en/content/getauthorversionpdf/C9CC05490A
Skipping invalid PDF at https://pubs.rsc.org/en/content/getauthorversionpdf/C9CC05490A
Merged PDF saved as '../data/pdfs/124.pdf'
https://doi.org/10.1039%2FC4TA05033A
https://pubs.rsc.org/en/content/articlelanding/2015/ta/c4ta05033a
https://pubs.rsc.org/
['/en/content/getauthorversionpdf/C4TA05033A']
https://pubs.rsc.org/en/content/getauthorversionpdf/C4TA05033A
Skipping invalid PDF at https://pubs.rsc.org/en/content/getauthorversionpdf/C4TA05033A
No PDF links found for index 125.
https://doi.org/10.1038%2Fs41563-018-0071-z
https://www.nature.com/artic

In [107]:
existing_files = {os.path.splitext(file)[0] for file in os.listdir('../data/pdfs')}
for index, row in merged_df.iterrows():
    index_str = str(index)
    if index_str not in existing_files:
        #showing which pdf prints were not successful (done manually for now, will be done with Selenium later)
        print(index)
        print(row['link'])

38
https://pubs.rsc.org/en/content/articlehtml/2019/ta/c9ta01070j
39
https://pubs.rsc.org/en/content/articlehtml/2019/ee/c9ee00453j
71
https://doi.org/10.48550/arXiv.2102.10399
74
https://doi.org/10.1146/annurev-food-022814-015651
76
https://doi.org/10.1039/C9CS00711C
82
https://doi.org/10.1039/TF937330008B
90
https://doi.org/10.1039%2FC7TA00434F
97
https://doi.org/10.1017/CBO9780511608810
100
https://doi.org/10.1146/annurev.bb.12.060183.001035
105
https://doi.org/10.1146/annurev.physchem.51.1.209
120
https://doi.org/10.1039/D1SM01707A
125
https://doi.org/10.1039%2FC4TA05033A
128
https://doi.org/10.1146/annurev.ms.12.080182.000535
129
https://doi.org/10.1039/DC9786500007


## Convert PDFs to XML using GROBID

In [10]:
grobid_url = "http://localhost:8070/api/processFulltextDocument"
xml_names = os.listdir("../data/xmls")

for pdf_file in os.listdir("../data/pdfs"):
    if pdf_file.endswith(".pdf"):
        pdf_path = os.path.join("../data/pdfs", pdf_file)
        if pdf_path.replace('.pdf', '.xml') in xml_names:
            continue
        with open(pdf_path, 'rb') as file:
            response = requests.post(
                grobid_url,
                files={'input': file},
                headers={'Accept': 'application/xml'}
            )

            if response.status_code == 200:
                xml_file_path = os.path.join('../data/xmls', pdf_file.replace('.pdf', '.xml'))
                with open(xml_file_path, 'w', encoding='utf-8') as xml_file:
                    xml_file.write(response.text)
                print(f"Converted {pdf_file} to XML.")
            else:
                print(f"Failed to convert {pdf_file}. Status code: {response.status_code}")
                print(response.text)

Converted 49.pdf to XML.
Converted 61.pdf to XML.
Converted 75.pdf to XML.
Converted 74.pdf to XML.
Converted 60.pdf to XML.
Converted 48.pdf to XML.
Converted 89.pdf to XML.
Converted 76.pdf to XML.
Converted 62.pdf to XML.
Converted 63.pdf to XML.
Converted 77.pdf to XML.
Converted 88.pdf to XML.
Converted 98.pdf to XML.
Converted 73.pdf to XML.
Converted 67.pdf to XML.
Converted 9.pdf to XML.
Converted 8.pdf to XML.
Converted 66.pdf to XML.
Converted 72.pdf to XML.
Converted 99.pdf to XML.
Converted 64.pdf to XML.
Converted 70.pdf to XML.
Converted 58.pdf to XML.
Converted 59.pdf to XML.
Converted 71.pdf to XML.
Converted 65.pdf to XML.


KeyboardInterrupt: 